In [ ]:
import pandas as pd
import json
import numpy as np
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import plotly.io as pio
# The bin folder has the DLLs
os.environ['path'] += r';C:/Users/JKIM4/Downloads/vips-dev-w64-all-8.11.0/vips-dev-8.11/bin'
import pyvips

In [ ]:
# setting directories
dir_code = "../run"
dir_data = dir_code + "/data"
dir_result = dir_code + "./results"

# setting configs json file path
file_configs = dir_code + "/configs.json"

# reading configs json file
print("reading configuration json file from = {}".format(file_configs))
with open(file_configs, "r") as read_file:
    configs = json.load(read_file)

configs['dir_code'] = dir_code
configs['dir_data'] = dir_data
configs['dir_result'] = dir_result
    
configs

In [ ]:
# reading FDD results file
df_result = pd.read_csv(configs["dir_result"] + "/{}_{}.csv".format(configs["weather"], configs["train_test_apply"]))
df_result

In [ ]:
# creating empty dataframe with timestamp
freq = str(configs['cost_est_timestep_min']) + 'min'
df_combined = pd.DataFrame([])
df_combined['reading_time'] = pd.date_range(configs['simulation_date_start'], configs['simulation_date_end'], freq=freq)
df_combined = df_combined.set_index(['reading_time'])[:-1]

df_combined

In [ ]:
# expanding FDD results into the same user-specified timestep
df_result_exp = np.repeat(list(df_result.values), configs['fdd_reporting_frequency_hrs']*int(60/configs['cost_est_timestep_min']))
df_result_exp = pd.DataFrame(df_result_exp)
df_result_exp.columns = ['FaultType']
df_result_exp.index = df_combined.index
df_result_exp

In [ ]:
# setting timestamp for raw simulation data
freq_raw = str(configs['simulation_timestep_min']) + 'min'
df_index = pd.DataFrame([])
df_index['reading_time'] = pd.date_range(configs['simulation_date_start'], configs['simulation_date_end'], freq=freq_raw)
df_index = df_index.set_index(['reading_time'])[:-1]
timestamp_last = df_index.index[-1]

In [ ]:
# reading baseline simulation results
print("[Estimating Fault Cost] reading baseline simulation results")
df_baseline = pd.read_csv(configs['dir_data']+"/"+configs['weather']+"/baseline.csv", usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
df_baseline.columns = [f"baseline_elec_{configs['sensor_unit_elec']}",f"baseline_ng_{configs['sensor_unit_ng']}"]
df_baseline.index = df_index.index
df_baseline = df_baseline.resample(str(configs['cost_est_timestep_min'])+"T").mean()
df_baseline

In [ ]:
# recreating FDD results with unique fault type (consecutive fault types are removed)
df_unique = df_result_exp[(df_result_exp.ne(df_result_exp.shift())).any(axis=1)]
df_unique = df_unique.reset_index()

# reading individual fault simulation results (based on FDD results) and creating whole year combined results
count = 1
print("[Estimating Fault Cost] combining simulation results from the FDD results")

df_combined_temp = pd.DataFrame()
for index, row in df_unique.iterrows():
    
    df_fault = pd.DataFrame()
    
    # specifying start and stop timestamp for each detected fault
    rownum_current = df_unique.loc[df_unique.index==index,:].index[0]
    timestamp_start = df_unique.iloc[rownum_current,:].reading_time
    if rownum_current+1 < df_unique.shape[0]:
        timestamp_end = df_unique.iloc[rownum_current+1,:].reading_time - pd.Timedelta(minutes=configs["simulation_timestep_min"])
    else:
        timestamp_end = timestamp_last
        
    print(f"[Estimating Fault Cost] prossessing [{row['FaultType']} ({count}/{df_unique.shape[0]})] from the FDD results covering {timestamp_start} to {timestamp_end}")
        
    count_file = 0
    for file in glob.glob(configs['dir_data']+"/"+configs['weather']+f"/*{row['FaultType']}*"):
        print(f"[Estimating Fault Cost] reading [{file}] file")
        count_file += 1
        if count_file == 1:
            df_temp = pd.read_csv(file, usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
            df_temp.index = df_index.index
            df_temp = df_temp.resample(str(configs['cost_est_timestep_min'])+"T").mean()
            df_fault = df_temp.copy()
        else:
            df_temp = pd.read_csv(file, usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
            df_temp.index = df_index.index
            df_temp = df_temp.resample(str(configs['cost_est_timestep_min'])+"T").mean()
            df_fault += df_temp
            
    # averaging all fault intensity simulations for a single fault and merging into combined dataframe
    print(f"[Estimating Fault Cost] averaging all fault intensity simulations for a single fault and merging into combined dataframe")
    df_fault = df_fault/count_file
    df_fault = df_fault[timestamp_start:timestamp_end]
    df_combined_temp = pd.concat([df_combined_temp, df_fault])
    count+=1
    
#     if count==3:
#         break

In [ ]:
df_combined_temp.to_csv("./df_combined_temp.csv")
# df_combined_temp = pd.read_csv("./df_combined_temp.csv", index_col=0)
df_combined_temp.columns = [f"faulted_elec_{configs['sensor_unit_elec']}",f"faulted_ng_{configs['sensor_unit_ng']}"]
df_combined_temp.index = pd.to_datetime(df_combined_temp.index)
df_combined_temp

In [ ]:
df_combined = pd.merge(df_combined, df_baseline, how='outer', left_index=True, right_index=True)
df_combined = pd.merge(df_combined, df_combined_temp, how='outer', left_index=True, right_index=True)
df_combined

In [ ]:
df_combined['diff_elec'] = df_combined["faulted_elec_{}".format(configs["sensor_unit_elec"])] - df_combined["baseline_elec_{}".format(configs["sensor_unit_elec"])]

In [ ]:
df_combined['diff_ng'] = df_combined["faulted_ng_{}".format(configs["sensor_unit_ng"])] - df_combined["baseline_ng_{}".format(configs["sensor_unit_ng"])]

In [ ]:
fig = make_subplots(
    rows=4, 
    cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.02,
    row_heights=[0.35, 0.15, 0.35, 0.15],
    # specs=[[{"secondary_y": True}],[{"secondary_y": True}]]
)

range_max_elec = max( df_combined["baseline_elec_{}".format(configs["sensor_unit_elec"])].max() , df_combined["faulted_elec_{}".format(configs["sensor_unit_elec"])].max() )
range_max_ng = max( df_combined["baseline_ng_{}".format(configs["sensor_unit_ng"])].max() , df_combined["faulted_ng_{}".format(configs["sensor_unit_ng"])].max() )
    
####################################################################
# plotting electricity
####################################################################

fig.add_trace(go.Scatter(
    x=df_combined.index,
    y=df_combined["baseline_elec_{}".format(configs["sensor_unit_elec"])].values,
    mode='lines',
    name="elec_baseline",
    #legendgroup = "electricity",
    showlegend=True,
    fill='tozeroy',
    opacity=0.1,
    line = dict(color='rgb(246,232,195)', width=1)
),
row=1, col=1, 
# secondary_y=False
)

fig.add_trace(go.Scatter(
    x=df_combined.index,
    y=df_combined["faulted_elec_{}".format(configs["sensor_unit_elec"])].values,
    mode='lines',
    name="elec_faulted",
    #legendgroup = "electricity",
    showlegend=True,
    fill='tonexty',
    opacity=1,
    line = dict(color='rgb(140,81,10)', width=0)
),
row=1, col=1, 
# secondary_y=False
)

fig.add_trace(go.Scatter(
    x=df_combined.index,
    y=df_combined["diff_elec"].values,
    mode='lines',
    name="elec_difference",
    #legendgroup = "electricity",
    showlegend=True,
    fill='tozeroy',
    opacity=1,
    line = dict(color='rgb(84,48,5)', width=0)
),
row=2, col=1, 
#secondary_y=True
)

####################################################################
# plotting natural gas
####################################################################

fig.add_trace(go.Scatter(
    x=df_combined.index,
    y=df_combined["baseline_ng_{}".format(configs["sensor_unit_ng"])].values,
    mode='lines',
    name="ng_baseline",
    #legendgroup = "natural gas",
    showlegend=True,
    fill='tozeroy',
    opacity=0.1,
    line = dict(color='rgb(199,234,229)', width=1)
),
row=3, col=1, 
# secondary_y=False
)

fig.add_trace(go.Scatter(
    x=df_combined.index,
    y=df_combined["faulted_ng_{}".format(configs["sensor_unit_ng"])].values,
    mode='lines',
    name="ng_faulted",
    #legendgroup = "natural gas",
    showlegend=True,
    fill='tonexty',
    opacity=1,
    line = dict(color='rgb(1,102,94)', width=0)
),
row=3, col=1, 
# secondary_y=False
)

fig.add_trace(go.Scatter(
    x=df_combined.index,
    y=df_combined["diff_ng"].values,
    mode='lines',
    name="ng_difference",
    #legendgroup = "natural gas",
    showlegend=True,
    fill='tozeroy',
    opacity=1,
    line = dict(color='rgb(0,60,48)', width=0)
),
row=4, col=1, 
# secondary_y=True
)

fig.update_layout(
    width=900,
    height=600,
    margin=dict(
        l=30,
        r=0,
        t=0,
        b=0,
    ),
    plot_bgcolor='white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5,
        font=dict(
            size=10,
        ),
    ),
)

fig.update_xaxes(
    title_text="<b>Time</b><br>",
    title_font_size = 12,
    row=4, col=1
)

fig.update_yaxes(title_text="Electricity<br>Consumption [{}]".format(configs["sensor_unit_elec"]), range=[0, range_max_elec*1.1], row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Natural Gas<br>Consumption [{}]".format(configs["sensor_unit_ng"]), range=[0, range_max_ng*1.1], row=3, col=1, secondary_y=False)

pio.write_image(fig, "./figures/fig_comparison_timeseries.svg")
# https://stackoverflow.com/questions/51450134/how-to-convert-svg-to-png-or-jpeg-in-python
image = pyvips.Image.thumbnail("./figures/fig_comparison_timeseries.svg", 3000)
image.write_to_file("./figures/fig_comparison_timeseries.png")

fig.show()

In [ ]:
#####################################################
# additional data processing
#####################################################

df_combined['Date'] = pd. to_datetime(df_combined.index).date
df_combined['Month'] = pd. to_datetime(df_combined.index).month
df_combined['Time'] = pd. to_datetime(df_combined.index).time
if (configs['sensor_unit_ng']=='W') & (configs['sensor_unit_elec']=='W'):
    df_monthly = df_combined.groupby(['Month'])[['diff_elec','diff_ng']].sum()/1000 # convert W to kW
    df_monthly = df_combined.groupby(['Month'])[['diff_elec','diff_ng']].sum()/(60/configs['cost_est_timestep_min']) #convert kW to kWh
    diff_annual_elec = round(df_monthly.sum()['diff_elec']) # in kWh
    diff_annual_ng = round(df_monthly.sum()['diff_ng']) # in kWh
else:
    # add other unit conversions
    print("[Estimating Fault Cost] unit conversion from {} for electricity and {} for natural gas to kWh is not currently supported".format(configs['sensor_unit_elec'],configs['sensor_unit_ng']))
    


#####################################################
# plot setting
#####################################################

linewidth = 0
linecolor = 'rgb(255,255,255)'
mirror = True
title_font_size = 10
colorbar_font_size = 10
tick_font_size = 10
anot_font_size = 20
fontfamily = 'verdana'
barwidth = 0.75
# colorscale=[
#     [0.0, 'rgb(5,48,97)'],
#     [0.1, 'rgb(33,102,172)'],
#     [0.2, 'rgb(67,147,195)'],
#     [0.3, 'rgb(146,197,222)'],
#     [0.4, 'rgb(209,229,240)'],
#     [0.5, 'rgb(247,247,247)'],
#     [0.6, 'rgb(253,219,199)'],
#     [0.7, 'rgb(244,165,130)'],
#     [0.8, 'rgb(214,96,77)'],
#     [0.9, 'rgb(178,24,43)'],
#     [1.0, 'rgb(103,0,31)']
# ]
colorscale=[
    [0.0, 'rgb(49,54,149)'],
    [0.1, 'rgb(69,117,180)'],
    [0.2, 'rgb(116,173,209)'],
    [0.3, 'rgb(171,217,233)'],
    [0.4, 'rgb(224,243,248)'],
    [0.5, 'rgb(255,242,204)'],
    [0.6, 'rgb(254,224,144)'],
    [0.7, 'rgb(253,174,97)'],
    [0.8, 'rgb(244,109,67)'],
    [0.9, 'rgb(215,48,39)'],
    [1.0, 'rgb(165,0,38)']
]

range_max_elec = max( df_combined['diff_elec'].max() , abs(df_combined['diff_elec'].min()) )
range_max_ng = max( df_combined['diff_ng'].max() , abs(df_combined['diff_ng'].min()) )


#####################################################
# plotting
#####################################################

fig = make_subplots(
    rows=2, 
    cols=3, 
    shared_xaxes=True, 
    vertical_spacing=0.025,
    horizontal_spacing=0.1,
    column_widths=[0.2, 0.4, 0.4],
    # specs=[[{"secondary_y": True}],[{"secondary_y": True}]]
)  

#####################################################
# heatmap
#####################################################

fig.add_trace(go.Heatmap(
    z=df_combined['diff_elec'],
    x=df_combined['Date'],
    y=df_combined['Time'],
    colorscale='tempo',
    coloraxis='coloraxis1',
),
row=1, col=3)

fig.add_trace(go.Heatmap(
    z=df_combined['diff_ng'],
    x=df_combined['Date'],
    y=df_combined['Time'],
    colorscale='tempo',
    coloraxis='coloraxis2',
),
row=2, col=3)

#####################################################
# bar chart
#####################################################

fig.add_trace(go.Bar(
    x=df_monthly.index,
    y=df_monthly['diff_elec'],
    showlegend=False,
    width=barwidth,
    marker=dict(
        cmid=0,
        color=df_monthly['diff_elec'],
        colorscale=colorscale,
    )
),
row=1, col=2)

fig.add_trace(go.Bar(
    x=df_monthly.index,
    y=df_monthly['diff_ng'],
    showlegend=False,
    width=barwidth,
    marker=dict(
        cmid=0,
        color=df_monthly['diff_ng'],
        colorscale=colorscale,
    )
),
row=2, col=2)

#####################################################
# annotation
#####################################################

fig.add_annotation(
    x=0.05,
    y=0.75,
    xref="paper",
    yref="paper",
    xanchor='center',
    yanchor='middle',
    text="Excess<br>electricity<br><b>{}<br>kWh/year</b>".format(diff_annual_elec),
    font=dict(
        family=fontfamily,
        size=anot_font_size,
        ),
    showarrow=False,
    align="center",
    )

fig.add_annotation(
    x=0.05,
    y=0.25,
    xref="paper",
    yref="paper",
    xanchor='center',
    yanchor='middle',
    text="Excess<br>natural gas<br><b>{}<br>kWh/year</b>".format(diff_annual_ng),
    font=dict(
        family=fontfamily,
        size=anot_font_size,
        ),
    showarrow=False,
    align="center",
    )

#####################################################
# layout
#####################################################

fig.update_layout(
#     title = dict( 
#         text="<b>Fault = thermostat bias</b>",
#         font=dict(
#             family=fontfamily,
#             size=title_font_size,
#         ),
#         x=0.5,
#         xanchor='center',
#         y=0.98,
#         yanchor='top',
#     ),
    width=900,
    height=400,
    margin=dict(
        l=0,
        r=0,
        t=0,
        b=0,
    ),
    plot_bgcolor='white',
    coloraxis1=dict(
        cmin=-range_max_elec,
        cmid=0,
        cmax=range_max_elec,
        colorscale=colorscale, 
        colorbar = dict(
            title=dict(
                text = "Excess electricty [{}]".format(configs['sensor_unit_elec']),
                side='right',
                font=dict(
                    size=colorbar_font_size,
                    family=fontfamily,
                ),
            ),
            len=0.5,
            x=1,
            xanchor='left',
            y=0.75,
            yanchor='middle',
            thickness=23,
        )
    ),
    coloraxis2=dict(
        cmin=-range_max_ng,
        cmid=0,
        cmax=range_max_ng,
        colorscale=colorscale, 
        colorbar = dict(
            title=dict(
                text = "Excess natural gas [{}]".format(configs['sensor_unit_ng']),
                side='right',
                font=dict(
                    size=colorbar_font_size,
                    family=fontfamily,
                ),
            ),
            len=0.5,
            x=1,
            xanchor='left',
            y=0.25,
            yanchor='middle',
            thickness=23,
        ),
    ),
)

#####################################################
# axes
#####################################################

fig.update_yaxes(
    showticklabels=False,
    row=2, col=1
)

fig.update_yaxes(
    showticklabels=False,
    row=1, col=1
)

fig.update_xaxes(
    title = dict( 
        text="<b>Date</b>",
        font=dict(
            family=fontfamily,
            size=title_font_size,
        ),
    ),
    tickfont = dict(
        family=fontfamily,
        size=tick_font_size,
    ),
    linecolor=linecolor,
    linewidth=linewidth,
    mirror=mirror,
    row=2, col=3
)

fig.update_xaxes(
    linecolor=linecolor,
    linewidth=linewidth,
    mirror=mirror,
    row=1, col=3
)

fig.update_xaxes(
    title = dict( 
        text="<b>Month</b>",
        font=dict(
            family=fontfamily,
            size=title_font_size,
        ),
    ),
    tickfont = dict(
        family=fontfamily,
        size=tick_font_size,
    ),
    dtick=1,
    linecolor=linecolor,
    linewidth=linewidth,
    mirror=mirror,
    row=2, col=2
)

fig.update_xaxes(
    dtick=1,
    linecolor=linecolor,
    linewidth=linewidth,
    mirror=mirror,
    row=1, col=2
)

fig.update_yaxes(
    title = dict( 
        text="<b>Time</b>",
        font=dict(
            family=fontfamily,
            size=title_font_size,
        ),
        standoff=0,
    ),
    tickfont = dict(
        family=fontfamily,
        size=tick_font_size,
    ),
    linecolor=linecolor,
    linewidth=linewidth,
    mirror=mirror,
    row=2, col=3
)

fig.update_yaxes(
    title = dict( 
        text="<b>Time</b>",
        font=dict(
            family=fontfamily,
            size=title_font_size,
        ),
        standoff=0,
    ),
    tickfont = dict(
        family=fontfamily,
        size=tick_font_size,
    ),
    linecolor=linecolor,
    linewidth=linewidth,
    mirror=mirror,
    row=1, col=3
)

fig.update_yaxes(
    title = dict( 
        text="<b>Electricity [kWh]</b>",
        font=dict(
            family=fontfamily,
            size=title_font_size,
        ),
        standoff=0,
    ),
    tickfont = dict(
        family=fontfamily,
        size=tick_font_size,
    ),
    linecolor=linecolor,
    linewidth=linewidth,
    mirror=mirror,
    row=1, col=2
)

fig.update_yaxes(
    title = dict( 
        text="<b>Natural gas [kWh]</b>",
        font=dict(
            family=fontfamily,
            size=title_font_size,
        ),
        standoff=0,
    ),
    tickfont = dict(
        family=fontfamily,
        size=tick_font_size,
    ),
    linecolor=linecolor,
    linewidth=linewidth,
    mirror=mirror,
    row=2, col=2
)

#####################################################
# export
#####################################################

pio.write_image(fig, "./figures/fig_comparison_heatmap.svg")
# https://stackoverflow.com/questions/51450134/how-to-convert-svg-to-png-or-jpeg-in-python
image = pyvips.Image.thumbnail("./figures/fig_comparison_heatmap.svg", 3000)
image.write_to_file("./figures/fig_comparison_heatmap.png")

fig.show()

In [ ]:
df_combined.groupby(['Month'])[['diff_elec','diff_ng']].sum()

In [ ]:
df_combined.to_csv("./output.csv")